# Run Pharaglow 

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
# image io and analysis
import json
import pims
import trackpy as tp
# plotting
import matplotlib  as mpl 
import matplotlib.pyplot as plt 

#our packages
from pharaglow import tracking, run, features

### Load data and create binary masks 

In [ ]:
%%time
# io
fname = "/media/scholz_la/hd2/Nicolina/Nicolina/20200109_NZ0017/*.tiff"
parameterfile = "/media/scholz_la/hd2/Nicolina/Nicolina/pharaglow_parameters_mks.txt"
outfile = "/media/scholz_la/hd2/Nicolina/Nicolina/20200109_NZ0017_{}_{}.json"
print('Starting pharaglow analysis...')
rawframes = pims.open(fname)
rawframes = rawframes[0:18000]
#print('Analyzing', rawframes)
print('Loading parameters from {}'.format(parameterfile.split('/')[-2:]))
with open(parameterfile) as f:
    param = json.load(f)


In [ ]:
%%time
# detecting objects
print('Binarizing images')
masks = tracking.calculateMask(rawframes, minSize = param['minSize'], bgWindow = 100, thresholdWindow = 100, smooth = 3, subtract = 1)


### Make sure the thesholding worked otherwise change parameters

In [ ]:
t = 1010
plt.figure(figsize=(12,11))
# plt.subplot(121)
# plt.imshow(rawframes[t])
# plt.subplot(122)
plt.imshow(masks[t])
#plt.imshow(masks[t][ 250:750, 1800:2000])
#plt.imshow(masks[t][1500:2000, 1000:2750])

#np.sum(masks[t][500:750, 1000:1500])
#print(np.sum(masks[t][ 350:750, 1600:2000]))

#image = masks[t][ 350:750, 1600:2000]
#labeled = tracking.refine(image, size = param['watershed'])
#plt.imshow(labeled)

### Detecting individual objects and tracking or use multiprocessing to speed up feature detection

In [ ]:
%%time
import time

nWorkers = 20
if nWorkers ==1:

    #masks = tracking.preprocess(rawframes, minSize = param['minSize'], threshold =None )
    print('Detecting features')
    features = tracking.runfeatureDetection(rawframes, masks, param, frameOffset = 0)
else:
    from multiprocessing import Pool
    print('Detecting features')
    def f(sl):
        a,b = sl
        #print(a,b)
        return tracking.runfeatureDetection(rawframes[a:b], masks[a:b], param, frameOffset = a)
    features = []
    L = len(rawframes)
    # create chunks of analysis based on how many workers we use
    print(L)
    chunksize = L//nWorkers//20
    #slices = np.arange(L)
    slices = zip((range(0,L, chunksize)), (range(chunksize,L+chunksize, chunksize)))
    
    p = Pool(20)
    start = time.time()
    for res in p.imap_unordered(f, slices):
        features.append(res)
        print(p, time.time()-start)
    features = pd.concat(features)

In [ ]:

### Save the features
plt.plot(np.sort(features['area'].values))

In [ ]:
features.head(5)
features.info(memory_usage='deep')
features.to_json(outfile.format('features', 'all'), orient='split')

In [ ]:
#features = pd.read_json(outfile.format('features'), orient='split', numpy = True)

In [ ]:
print('Done')
print('Linking trajectories')
#trajectories = tracking.linkParticles(features, param['searchRange'], param['minimalDuration'])
#trajectories = tracking.linkParticles(features, 50, 500);
#trajectories = tp.link_df(features, 50, memory= 30)
#trajectories = tp.filter_stubs(trajectories,100)
pred = tp.predict.NearestVelocityPredict()
trajectories = pred.link_df(features, 40, memory= 300)

In [ ]:
print(trajectories['particle'].nunique())
trajectories = tp.filter_stubs(trajectories,500)
print(trajectories['particle'].nunique())

### Show resulting trajectories

In [ ]:
plt.figure(figsize=(11,11))
tp.plot_traj(trajectories, superimpose=1-masks[900]);

In [ ]:
from multiprocessing import Pool
def parallelize_dataframe(df, func, params, n_cores):
    df_split = np.array_split(df, n_cores)
    print([len(d) for d in df_split])
    pool = Pool(n_cores)
    df = pd.concat(pool.starmap(func, zip(df_split, np.repeat(params, len(df_split)))))
    pool.close()
    pool.join()
    return df

### run the whole pharaglow feature extraction

In [ ]:
%%time
print('Extracting pharynx data')
trajectories = parallelize_dataframe(trajectories, run.runPharaglowOnStack, n_cores = 20, params = param)
print('Done tracking. Successfully tracked {} frames with {} trajectorie(s).'.format(len(rawframes), trajectories['particle'].nunique()))

### Save data as hdf5 format (every trajectory in a file)

In [ ]:
#trajectories.info(memory_usage='deep')
for particle_index in trajectories['particle'].unique():
    trajectories[trajectories.loc[:,'particle'] == particle_index].to_json(outfile.format('results', particle_index), orient='split')